<a href="https://colab.research.google.com/github/jiananyang81/QMSS-5074/blob/main/assignment%20%233.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

link to Github repo:https://github.com/jiananyang81/QMSS-5074

Part I: Use the IMDB LSTM notebook from last week's class to access data and to create your own training and test dataset.

Discuss the dataset in general terms and describe why building a predictive model using this data might be practically useful.  Who could benefit from a model like this? Explain.

IMDB dataset having 50K movie reviews for natural language processing or Text analytics. This is a dataset for binary sentiment classification containing substantially more data. They provide a set of 25,000 highly polar movie reviews for training and 25,000 for testing. So, predict the number of positive and negative reviews using either classification or deep learning algorithms is possible. This model could be advantageous for companies. By examining the sentiment of movie reviews, companies can pinpoint areas that need improvement and make informed decisions regarding their marketing strategies. Additionally, customers can also benefit from this model. They can use the predictions to determine which movies to watch and which ones to avoid.

Run at least three prediction models to try to predict the IMDB sentiment dataset well.
Use an Embedding layer and LSTM layers in at least one model
Use an Embedding layer and Conv1d layers in at least one model
Use transfer learning with glove embeddings for at least one of these models
Discuss which models performed better and point out relevant hyper-parameter values for successful models


In [ ]:
# Get raw imdb dataset
! wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

--2023-12-16 18:37:03--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz.1’

aclImdb_v1.tar.gz.1 100%[===================>]  80.23M  17.5MB/s    in 7.5s    

2023-12-16 18:37:10 (10.8 MB/s) - ‘aclImdb_v1.tar.gz.1’ saved [84125825/84125825]



In [ ]:
# Untar it to a new folder
! tar xf aclImdb_v1.tar.gz

In [ ]:
# Build corpus of docs and labels
import os

imdb_dir = 'aclImdb'
train_dir = os.path.join(imdb_dir, 'train')

labels = []
texts = []

for label_type in ['neg', 'pos']:
    dir_name = os.path.join(train_dir, label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname))
            texts.append(f.read())
            f.close()
            if label_type == 'neg':
                labels.append(0)
            else:
                labels.append(1)

In [ ]:
len(texts)

25000

In [ ]:
print(texts[0])
print(labels[0])

This movie tries to hard to be something that it's not....a good movie. It wants you to be fooled from begining to end,But fails.From when it starts to get interesting it falls apart and you're just hoping the ending gives you some clue of just what is going on but it didn't.<br /><br />
0


In [ ]:
# Tokenize the data into one hot vectors
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

maxlen = 100  # We will cut reviews after 100 words in sequence
training_samples = 10000  # We will be training on 10000 samples
validation_samples = 10000  # We will be validating on 10000 samples
max_words = 10000  # We will only consider the top 10,000 words in the dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)


sequences = tokenizer.texts_to_sequences(texts) # converts words in each text to each word's numeric index in tokenizer dictionary.

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=maxlen)

labels = np.asarray(labels)

print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# Split the data into a training set and a validation set
# But first, shuffle the data, since we started from data
# where sample are ordered (all negative first, then all positive).
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

x_train = data[:training_samples] #100 words
y_train = labels[:training_samples]
x_val = data[training_samples: training_samples + validation_samples]
y_val = labels[training_samples: training_samples + validation_samples]

Found 88582 unique tokens.
Shape of data tensor: (25000, 100)
Shape of label tensor: (25000,)


In [ ]:
#Text example
print(texts[0])

#Text transformed to sequence using tokenizer
print(sequences[0])

This movie tries to hard to be something that it's not....a good movie. It wants you to be fooled from begining to end,But fails.From when it starts to get interesting it falls apart and you're just hoping the ending gives you some clue of just what is going on but it didn't.<br /><br />
[11, 17, 494, 5, 251, 5, 27, 139, 12, 42, 21, 3, 49, 17, 9, 490, 22, 5, 27, 4449, 36, 5, 127, 18, 993, 36, 51, 9, 514, 5, 76, 218, 9, 731, 968, 2, 332, 40, 1379, 1, 274, 405, 22, 46, 2297, 4, 40, 48, 6, 167, 20, 18, 9, 158, 7, 7]


In [ ]:
# sequences preprocessed with tokenizer with zeroes added whenver text isn't 100 words long.
data[0]

array([  71,  230,  184,   87,   28,  248,   24,  319, 1065,    5, 8090,
          1,  516,  234,   10,   89,  178,    5, 2368,   21,   12,   47,
         13,  230,    5, 2368,   10, 3422,  384,  126, 1542,  819,   43,
          1, 9019,    2,   10,  358,   12,  678, 9060, 3888,   12,  791,
          7,    7,   10,   66,    5,  545,    5,  785, 4160,  140,   11,
          3,  783, 8903,    4,    3,   19,    2,   10,   59,   37,    5,
        132,   12,   44,   22,   23,    5,    1,   17, 1129,    2,   23,
        533,   41,  109, 9545,  588,   89,   42,    3,  434,    4,    1,
         19,    9,   13,   20,    7,    7,   92,  171,   10,   97,   27,
        352], dtype=int32)

Use transfer learning with glove embeddings for at least one of these models

In [ ]:
# Let's start with a model that ignores the sequential steps that make up each observation
from tensorflow.keras.layers import Dense, Embedding,Flatten
from tensorflow.keras.models import Sequential

model = Sequential()
# Specify the size of your vocabulary (i.e.-10,000 terms)
# Specify the number of features you want to extract via fitting weights to your embedding matrix.
# We also specify the maximum input length to our Embedding layer
# so we can later flatten the embedded inputs
model.add(Embedding(10000, 16, input_length=maxlen))
# After the Embedding layer,
# our activations have shape `(samples, maxlen, 8)`.

# We flatten the 3D tensor of embeddings
# into a 2D tensor of shape `(samples, maxlen * 8)`
model.add(Flatten())

# We add the classifier on top
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.summary()

history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 100, 16)           160000    
                                                                 
 flatten_4 (Flatten)         (None, 1600)              0         
                                                                 
 dense_4 (Dense)             (None, 1)                 1601      
                                                                 
Total params: 161601 (631.25 KB)
Trainable params: 161601 (631.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
250/250 [==============================] - 2s 8ms/step - loss: 0.6937 - acc: 0.4970 - val_loss: 0.6926 - val_acc: 0.5190
Epoch 2/10
250/250 [==============================] - 2s 6ms/step - loss: 0.6703 - acc: 0.7315 - val_loss: 0.6925 - val_acc: 0.5230
Epoch 3/10
250

In [ ]:
# What does the output of the embedding layer look like?  (It returns a sequentially ordered transformation of numerically indexed input word data)
# Credit: Code adjusted from ML Mastery

import tensorflow as tf

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(10000, 8, input_length=100))
# The model will take as input an integer matrix of size (batch,
# input_length).
# Now model.output_shape is (None, 100, 8), where `None` is the batch
# dimension.
input_array = np.random.randint(1000, size=(1, 100))
model.compile('rmsprop', 'mse')
output_array = model.predict(input_array)
print(output_array.shape)
print(output_array[0])

1/1 [==============================] - 0s 164ms/step
(1, 100, 8)
[[-0.04039253 -0.04411009 -0.03234978  0.01725081  0.0454383   0.02619984
  -0.03623825  0.00021284]
 [ 0.00197277  0.04725694 -0.02286728  0.00680789  0.0391122   0.03458827
  -0.03983982 -0.01803257]
 [ 0.01051817 -0.0213825   0.03032181 -0.00964444  0.0290267  -0.03048908
   0.01417166 -0.037655  ]
 [-0.03853192  0.03989482 -0.03746786 -0.0452791  -0.02051405 -0.0036217
  -0.0175675   0.03449437]
 [-0.02627106 -0.03013691  0.00368441 -0.0293501  -0.00521041  0.02580566
  -0.03471079 -0.04385408]
 [ 0.00995962  0.00254407  0.03331595 -0.02559943  0.02167854 -0.04757075
   0.04240945  0.02990988]
 [ 0.0421157   0.01651325  0.01130859 -0.02158054  0.01287824 -0.03106356
  -0.03269391 -0.01969727]
 [-0.0143639   0.01808712  0.00882574 -0.03894784 -0.02497778  0.00896768
   0.01192253 -0.02855209]
 [-0.00175601  0.04915081  0.01583591 -0.00799948 -0.01359546 -0.00654449
  -0.04536191  0.01702083]
 [-0.00455693  0.02848324 -

In [ ]:
! wget http://nlp.stanford.edu/data/wordvecs/glove.6B.zip

--2023-12-16 18:58:30--  http://nlp.stanford.edu/data/wordvecs/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/wordvecs/glove.6B.zip [following]
--2023-12-16 18:58:30--  https://nlp.stanford.edu/data/wordvecs/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/wordvecs/glove.6B.zip [following]
--2023-12-16 18:58:31--  https://downloads.cs.stanford.edu/nlp/data/wordvecs/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182753 (822M) [app

In [ ]:
! unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       
  inflating: glove.6B.50d.txt        


In [ ]:
# Extract embedding data for 100 feature embedding matrix
glove_dir = os.getcwd()

embeddings_index = {}
f = open(os.path.join(glove_dir, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400001 word vectors.


In [ ]:
# Build embedding matrix
embedding_dim = 100 # change if you use txt files using larger number of features

embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if i < max_words:
        if embedding_vector is not None:
            # Words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector

In [ ]:
# Set up same model architecture as before and then import Glove weights to Embedding layer:

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(max_words, embedding_dim, input_length=maxlen))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.summary()



Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 100, 100)          1000000   
                                                                 
 flatten_5 (Flatten)         (None, 10000)             0         
                                                                 
 dense_5 (Dense)             (None, 32)                320032    
                                                                 
 dense_6 (Dense)             (None, 1)                 33        
                                                                 
Total params: 1320065 (5.04 MB)
Trainable params: 1320065 (5.04 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:

# Add weights in same manner as transfer learning and turn of trainable option before fitting model to freeze weights.
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False



model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])
history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_data=(x_val, y_val))
model.save_weights('pre_trained_glove_model.h5')


# Training data small to speed up training. Increase for better fit.


Epoch 1/10
313/313 [==============================] - 4s 10ms/step - loss: 0.7288 - acc: 0.5072 - val_loss: 0.6933 - val_acc: 0.4992
Epoch 2/10
313/313 [==============================] - 2s 8ms/step - loss: 0.6984 - acc: 0.5097 - val_loss: 0.6933 - val_acc: 0.5000
Epoch 3/10
313/313 [==============================] - 2s 8ms/step - loss: 0.6928 - acc: 0.5183 - val_loss: 0.6941 - val_acc: 0.5006
Epoch 4/10
313/313 [==============================] - 4s 12ms/step - loss: 0.6822 - acc: 0.5457 - val_loss: 0.7009 - val_acc: 0.5044
Epoch 5/10
313/313 [==============================] - 4s 11ms/step - loss: 0.6590 - acc: 0.5922 - val_loss: 0.7092 - val_acc: 0.5019
Epoch 6/10
313/313 [==============================] - 3s 10ms/step - loss: 0.6079 - acc: 0.6466 - val_loss: 0.7644 - val_acc: 0.4955
Epoch 7/10
313/313 [==============================] - 3s 10ms/step - loss: 0.5419 - acc: 0.7124 - val_loss: 0.8019 - val_acc: 0.5011
Epoch 8/10
313/313 [==============================] - 3s 10ms/step - lo

In [ ]:
# Evaluate model on test set (need to preprocess test data to same structure first)

test_dir = os.path.join(imdb_dir, 'test')

labels = []
texts = []

for label_type in ['neg', 'pos']:
    dir_name = os.path.join(test_dir, label_type)
    for fname in sorted(os.listdir(dir_name)):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname))
            texts.append(f.read())
            f.close()
            if label_type == 'neg':
                labels.append(0)
            else:
                labels.append(1)

#using tokenizer object we fit to test data above
sequences = tokenizer.texts_to_sequences(texts)
x_test = pad_sequences(sequences, maxlen=maxlen)
y_test = np.asarray(labels)

In [ ]:
model.load_weights('pre_trained_glove_model.h5')
model.evaluate(x_test, y_test)



782/782 [==============================] - 2s 3ms/step - loss: 1.2172 - acc: 0.4923


[1.2171597480773926, 0.4923200011253357]

Use an Embedding layer and LSTM layers in at least one model

In [ ]:
model2 = Sequential()
model2.add(Embedding(10000, 32, input_length=maxlen))
model2.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model2.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs

model2.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model2.fit(x_train, y_train,
                    epochs=1,
                    batch_size=32,
                    validation_split=0.2)

250/250 [==============================] - 65s 250ms/step - loss: 0.6936 - acc: 0.4965 - val_loss: 0.6933 - val_acc: 0.4850


In [ ]:
score, acc = model2.evaluate(x_test, y_test)
print('Test score:', score)
print('Test accuracy:', acc)

782/782 [==============================] - 32s 41ms/step - loss: 0.6932 - acc: 0.4963
Test score: 0.6932473182678223
Test accuracy: 0.4963200092315674


Use an Embedding layer and Conv1d layers in at least one model

In [ ]:
# Use 1D Conv layer rather than RNN or LSTM or GRU to fit model
# Why? Much lighter model to fit. Here we are training on the full dataset.  If you try
# to build a model using LSTM code after running this one it will be much slower.

from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import SimpleRNN, LSTM,Embedding

model3 = Sequential()
model3.add(layers.Embedding(10000, 8, input_length=maxlen))
model3.add(layers.Conv1D(32, 7, activation='relu'))
model3.add(layers.MaxPooling1D(5)) #
model3.add(layers.Conv1D(32, 7, activation='relu'))
model3.add(layers.GlobalMaxPooling1D())
model3.add(layers.Dense(1))

model3.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    (None, 100, 8)            80000     
                                                                 
 conv1d_2 (Conv1D)           (None, 94, 32)            1824      
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 18, 32)            0         
 g1D)                                                            
                                                                 
 conv1d_3 (Conv1D)           (None, 12, 32)            7200      
                                                                 
 global_max_pooling1d_1 (Gl  (None, 32)                0         
 obalMaxPooling1D)                                               
                                                                 
 dense_10 (Dense)            (None, 1)               

In [ ]:
model3.compile(optimizer=RMSprop(lr=1e-4),
              loss='binary_crossentropy',
              metrics=['acc'])
history = model3.fit(x_train, y_train,
                    epochs=1,
                    batch_size=128,
                    validation_split=0.2)

63/63 [==============================] - 3s 24ms/step - loss: 7.7414 - acc: 0.4981 - val_loss: 7.8667 - val_acc: 0.4900


In [ ]:
score, acc = model3.evaluate(x_test, y_test)
print('Test score:', score)
print('Test accuracy:', acc)

782/782 [==============================] - 2s 3ms/step - loss: 7.7125 - acc: 0.5000
Test score: 7.712466239929199
Test accuracy: 0.5


The embedding layer and Conv1d layers perform better.